* Colab jupyter notebook끼리 import하기 위한 선언

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 2

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 2


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os

In [3]:
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 2/sample')

In [4]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 2/sample']

### 자연어와 단어의 분산 표현
- 시소러스를 활용한 기법
- 통계 기반 기법
- 추론 기반 기법

### 시소러스
 - 사람이 직접 단어의 의미를 정의하는 방식
 - WordNet 대표적 시소러스
 - 단어사이의 유사도를 구할 수 있음

단점
- 시대변화에 대응하기 어려움 (ex) 신조어, 의미변화 단어
- 인력비용이 큼
- 단어의 미묘한 차이를 표현할 수 없음


### 통계 기반 기법
- 말뭉치에서 자동으로, 효율적으로 그 핵심을 추출하는 것

텍스트를 단어로 분할 -> 단어를 ID목록으로 변환

In [8]:
text = "You say goodbye and I say hello."

In [9]:
text = text.lower()
text = text.replace('.',' .')
print(text)

you say goodbye and i say hello .


In [11]:
words = text.split(' ')
print(words)

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']


In [13]:
word_to_id = {}
id_to_word = {}

for word in words:
  if word not in word_to_id:
    new_id = len(word_to_id)
    word_to_id[word] = new_id
    id_to_word[new_id] = word

In [14]:
word_to_id

{'.': 6, 'and': 3, 'goodbye': 2, 'hello': 5, 'i': 4, 'say': 1, 'you': 0}

In [15]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [16]:
id_to_word[1]

'say'

In [17]:
corpus = [word_to_id[w] for w in words]
pritn(corpus)

[0, 1, 2, 3, 4, 1, 5, 6]

In [18]:
def preprocess(text):
  text = text.lower()
  text = text.replace('.',' .')
  words = text.split(' ')
  word_to_id = {}
  id_to_word = {}

  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id] = word

  corpus = np.array([word_to_id[w] for w in words])

  return corpus, word_to_id, id_to_word

In [19]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)

* 우리가 구할 것은 '단어의 의미'를 정확하게 파악할 수 있는 벡터 표현

In [20]:
# 분포가설 : 단어의 의미는 주변 단어에 의해 형성된다.
# 맥락이 의미를 형성한다.

In [21]:
from sample.common.util import preprocess

In [22]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)

In [23]:
print(corpus)
print(word_to_id)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [25]:
# 동시발생 행렬 만들어주는 함수 구현
def create_co_matrix(corpus, vocab_size, window_size=1):
  corpus_size = len(corpus)
  co_matrix = np.zeros((vocab_size, vocab_size), dtype=int32)

  for idx, word_id in enmuerate(corpus):
    for i in range(1, window_size+1):
      left_idx = idx-1
      right_idx = idx+1

      if left_idx >= 0:
        left_word_id = corpus[left_idx]
        co_matrix[word_id, left_word_id] += 1

      if right_idx < corpus_size:
        right_word_id = corpus[right_idx]
        co_matrix[word_id, right_word_id] += 1

  return co_matrix

In [27]:
# 코사인 유사도 함수
def cos_similarity(x, y, eps=1e-8):
  nx = x / np.sqrt(np.sum(x**2) + eps) #정규화 과정
  ny = y / np.sqrt(np.sum(y**2) + eps) #정규화 과정
  return np.dot(nx, ny)

In [28]:
from sample.common.util import preprocess, cos_similarity, create_co_matrix

In [29]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

In [30]:
C

array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]], dtype=int32)

In [31]:
c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
print(cos_similarity(c0, c1))

0.7071067691154799


In [32]:
#유사 단어의 랭킹 표시
def most_similarity(query, word_to_id, id_to_word, word_matrix, top=5):
  if query not in word_to_id:
    print("{} 를 찾을 수 없습니다.".format(query))
    return

  print(query)
  query_id = word_to_id[query]
  query_vec = word_matrix[query_id]

  vocab_size = len(id_to_word)
  similarity = np.zeros(vocab_size)
  for i in range(vocab_size):
    similarity[i] = cos_similarity(word_matrix[i], query_vec)
  
  count = 0
  for i in (-1 * similarity).argsort():
    if id_to_word[i] == query:
      continue
    print(id_to_word[i], similarity[i])
    
    count += 1
    if count >= top:
      return 

In [33]:
x = np.array([100, -20, 2])
np.argsort(x)

array([1, 2, 0])

In [36]:
from sample.common.util import preprocess, cos_similarity, create_co_matrix, most_similar

text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0
